In [9]:
# IMPORT REQUIRED PACKAGES
import duckdb
import pandas as pd
import os
import statistics
import re

## Load data

### Duck DB connector and data loader

In [10]:
# DUCKDB CONNECTOR
conn = duckdb.connect()

# Create datasets
def load_data():
    '''Loading data from online review csv files'''
    data_list = []
    for file in os.listdir('./GoogleReviews/'):
        if file.endswith('.csv'):
            
            file = './GoogleReviews/'+file
            data_list.append(file)
            
    return data_list


In [16]:
# Load data from GoogleReviews map
review_datasets = load_data()

print(review_datasets)

['./GoogleReviews/reviews_Ellis_ChIJ___y-8YJxkcRazeKOde9ty8.csv', './GoogleReviews/reviews_Samar_ChIJ4WvMT3fJCTMRhfrmlLIn0zY.csv', './GoogleReviews/reviews_House of Watt_ChIJjbCuMnkJxkcRQlR2ZPaKhrg.csv', './GoogleReviews/reviews_Zero20 Amsterdam_ChIJ__-zE-kJxkcRwjzVmrUPGFg.csv', './GoogleReviews/reviews_McDonald_s_ChIJsVu5gnMJxkcRAETTOEVHxMw.csv', './GoogleReviews/reviews_Firma Pekelharing_ChIJm0D-qI8JxkcRco7cqjacLyg.csv', './GoogleReviews/reviews_Bredero_s Cafe_ChIJc1JXdOl1sEcRPLJsvqkOlQQ.csv', './GoogleReviews/reviews_Singel_ChIJYVeJQcYJxkcROcbyKp8xGrk.csv', './GoogleReviews/reviews_Café Del Mondo_ChIJn0zxL7kJxkcRmnTX-tghMZE.csv', './GoogleReviews/reviews_café de Groene Vlinder_ChIJYUv3UY0JxkcR1ZUeaJNZb9Y.csv', './GoogleReviews/reviews_FEBO Amsteldijk_ChIJAcwrK4EJxkcR7OHjxG-qQ9k.csv', './GoogleReviews/reviews_Gordion Eethuis_ChIJfeNG2PAIxkcRwpOqftQwLRk.csv', './GoogleReviews/reviews_Sushi_N_Sake_ChIJ86nRbJsF2UcR8VkOB6yYcJQ.csv', './GoogleReviews/reviews_Locanda_ChIJLcJ0_s8JxkcRGJVegy

In [ ]:
# Create a list of only place_ids
review_placeid = []

for rev in review_datasets:
    review_placeid.append('')

In [13]:
# List of tables
table_names = []
for rev in review_datasets:
    table_names.append('table{0}'.format(review_datasets.index(rev)))

table_names_clean = ['clean_table']

joined_tables = []
for rev in review_datasets:
    joined_tables.append('joined_table{0}'.format(review_datasets.index(rev)))

### Create tables for each file

In [42]:
def create_table(datasets, table_names):
    '''Create tables from data.
        NB! Format of datasets and table_names needs to be list of equal length'''
    for rev in datasets:
        try:
            conn.execute('''CREATE TABLE {0} AS SELECT * FROM read_csv_auto ('{1}')'''.format(table_names[datasets.index(rev)], rev))
        except:
            continue
        
    return

In [43]:
# CREATE TABLES
create_table(review_datasets, table_names)

In [44]:
# Length of data
len(table_names)

1948

In [52]:
# View table
conn.execute("SELECT * FROM table204").fetch_df()

,index,name,Rating,Date,text
0,NaN,Name,Review Rate,Review Time,Review Text
1,0.0,de Veranda,5 stars,4 months ago,Had been to this place with colleagues and cli...
2,1.0,de Veranda,5 stars,2 weeks ago,The food was absolutely delicious and meals ar...
3,2.0,de Veranda,5 stars,8 months ago,"Great service, nice atmosphere and a delicious..."
4,3.0,de Veranda,5 stars,a year ago,Amazing environment and food! One tip: Reserve...
...,...,...,...,...,...
926,925.0,de Veranda,4 stars,2 years ago,NaN
927,926.0,de Veranda,4 stars,2 years ago,NaN
928,927.0,de Veranda,5 stars,4 years ago,NaN
929,928.0,de Veranda,5 stars,2 years ago,NaN


In [54]:
def column_names(table_names):
    for t in table_names:
        try:
            conn.execute("ALTER TABLE {0} RENAME column0 to index".format(t))
            conn.execute("ALTER TABLE {0} RENAME column1 to Name".format(t))
            conn.execute("ALTER TABLE {0} RENAME column2 to Rating".format(t))
            conn.execute("ALTER TABLE {0} RENAME column3 to Date".format(t))
            conn.execute("ALTER TABLE {0} RENAME column4 to Text".format(t))
        except:
            continue
        
    return
        

In [55]:
column_names(table_names)

## Clean data

In [26]:
def remove_punctuation(text):
    text = str(text)
    text = text.lower()
    raw_text = re.sub(u"[àáâãäåæ]", 'a', x)
    raw_text = re.sub(u"[èéêë]", 'e', raw_text)
    raw_text = re.sub(u"[ìíîï]", 'i', raw_text)
    raw_text = re.sub(u"[òóôõöøớ]", 'o', raw_text)
    raw_text = re.sub(u"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(u"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(u"[ß]", 'ss', raw_text)
    raw_text = re.sub(u"[ñ]", 'n', raw_text)
    raw_text = re.sub(u"[ç]", 'c', raw_text)
    raw_text = re.sub(u"[_½]", '', raw_text)
    raw_text = re.sub(u"[  ]", ' ', raw_text)
    raw_text = re.sub(r'[^\w\s]', '', raw_text)
    raw_text = raw_text.replace('  ', ' ')
    return raw_text